# PhoWhisper-large ASR (Kaggle ổn định, không “đứng im”)

Notebook này là bản sửa hoàn chỉnh từ code của bạn:

- Giữ nguyên model: `vinai/PhoWhisper-large`
- Tắt Xet/CAS, tải model chắc chắn bằng `snapshot_download` + retry
- Load **mỗi GPU 1 model riêng** (multi-GPU đúng cách)
- Pad/cắt mel về **3000** (Whisper yêu cầu)
- Auto-split batch khi OOM
- Lưu JSON + checkpoint tạm `_{Lxx}.partial.json` để không mất tiến trình

> Lần đầu có thể lâu vì tải model; những lần sau sẽ nhanh do dùng cache.


In [1]:
# =========================
# (0) Cài thư viện (khuyến nghị)
# =========================
!pip -q install -U "transformers==4.41.2" "huggingface_hub==0.24.7" "accelerate==0.33.0" "protobuf==4.25.3" natsort tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
# =========================
# (1) CONFIG
# =========================
import os, sys, time, json
import torch
import torchaudio
from tqdm import tqdm
from natsort import natsorted
from concurrent.futures import ThreadPoolExecutor, as_completed

WAV_ROOT     = "/kaggle/input/wav-scene/wav_scenes"    # thư mục chứa L01, L02, ...
OUTPUT_ROOT  = "/kaggle/working/asr_json"             # output json mỗi Lxx.json
MODEL_ID     = "vinai/PhoWhisper-large"               # ✅ giữ nguyên
RUN_FOLDER   = 24                                      # 6 -> L06 ; None -> chạy tất cả
BATCH_SIZE   = 16                                     # nếu hay OOM: 8 hoặc 4

# ép 2 GPU nếu bạn có (tự clamp theo số GPU hiện có)
FORCE_GPUS   = 2   # None để auto

# --------------------------
# môi trường ổn định
# --------------------------
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")
os.environ["HF_HUB_DISABLE_XET"] = "1"  # tránh lỗi 401/treo do Xet/CAS

HF_TOKEN = os.environ.get("HF_TOKEN", None)  # set trong Kaggle Secrets nếu cần

os.makedirs(OUTPUT_ROOT, exist_ok=True)

def log(msg: str):
    print(f"[{time.strftime('%H:%M:%S')}] {msg}")
    sys.stdout.flush()

log("Config loaded.")


[19:57:35] Config loaded.


In [3]:
# =========================
# (2) Download model chắc chắn (retry) → tránh treo / đứng im
# =========================
from huggingface_hub import snapshot_download

def snapshot_download_if_needed(model_id: str, token=None, tries=5):
    for i in range(1, tries+1):
        try:
            log(f"snapshot_download try {i}/{tries} for {model_id} ...")
            snapshot_download(repo_id=model_id, token=token, resume_download=True)
            log("✅ snapshot_download done.")
            return
        except Exception as e:
            log(f"⚠️ snapshot_download failed: {type(e).__name__}: {e}")
            if i == tries:
                raise
            time.sleep(min(30, 2**i))

snapshot_download_if_needed(MODEL_ID, token=HF_TOKEN, tries=5)


[19:57:35] snapshot_download try 1/5 for vinai/PhoWhisper-large ...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

README.md: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

[19:58:10] ✅ snapshot_download done.


In [4]:
# =========================
# (3) Load processor + load 1 model riêng cho mỗi GPU
# =========================
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

detected = torch.cuda.device_count()
if detected == 0:
    raise RuntimeError("Không thấy GPU. Bật GPU trong Kaggle Settings → Accelerator.")

num_gpus = detected if FORCE_GPUS is None else min(FORCE_GPUS, detected)
if FORCE_GPUS is not None and detected < FORCE_GPUS:
    log(f"⚠️ FORCE_GPUS={FORCE_GPUS} nhưng chỉ thấy {detected} GPU. Dùng {num_gpus} GPU.")

log("📥 Loading processor (local_files_only=True)...")
processor = AutoProcessor.from_pretrained(MODEL_ID, token=HF_TOKEN, local_files_only=True)
log("✅ Processor loaded.")

gpu_models = {}
for gpu_id in range(num_gpus):
    log(f" → Loading model on cuda:{gpu_id}")
    gpu_models[gpu_id] = AutoModelForSpeechSeq2Seq.from_pretrained(
        MODEL_ID,
        token=HF_TOKEN,
        local_files_only=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
    ).to(f"cuda:{gpu_id}").eval()

log(f"✅ Loaded {num_gpus} model(s): {list(gpu_models.keys())}")


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

2025-12-03 19:58:13.368429: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764791893.767746      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764791893.882962      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[19:58:39] 📥 Loading processor (local_files_only=True)...
[19:58:39] ✅ Processor loaded.
[19:58:39]  → Loading model on cuda:0
[19:58:46]  → Loading model on cuda:1
[19:58:50] ✅ Loaded 2 model(s): [0, 1]


In [5]:
# =========================
# (4) Hàm ASR (pad/cắt 3000 + auto-split khi OOM + checkpoint)
# =========================
def load_audio_16k_mono(wav_path: str):
    wave, sr = torchaudio.load(wav_path)
    if wave.ndim == 2 and wave.shape[0] > 1:
        wave = wave.mean(dim=0, keepdim=True)
    if sr != 16000:
        wave = torchaudio.functional.resample(wave, sr, 16000)
    return wave.squeeze().numpy()

def pad_trim_to_3000(input_features: torch.Tensor) -> torch.Tensor:
    target = 3000
    cur = input_features.shape[-1]
    if cur < target:
        pad_len = target - cur
        pad = torch.zeros(
            input_features.shape[0], input_features.shape[1], pad_len,
            device=input_features.device, dtype=input_features.dtype
        )
        input_features = torch.cat([input_features, pad], dim=-1)
    elif cur > target:
        input_features = input_features[:, :, :target]
    return input_features

def transcribe_batch(gpu_id: int, wav_batch: list[str], depth=0):
    """Return texts aligned with wav_batch length. On OOM, split batch and retry."""
    try:
        waveforms = []
        valid_indices = []
        for idx, wav_path in enumerate(wav_batch):
            try:
                waveforms.append(load_audio_16k_mono(wav_path))
                valid_indices.append(idx)
            except Exception as e:
                log(f"⚠️ Load fail: {os.path.basename(wav_path)} -> {e}")
                waveforms.append(None)

        valid_waveforms = [w for w in waveforms if w is not None]
        if not valid_waveforms:
            return ["" for _ in wav_batch]

        inputs = processor(
            valid_waveforms,
            sampling_rate=16000,
            return_tensors="pt",
            padding=True
        )

        device = torch.device(f"cuda:{gpu_id}")
        input_features = inputs.input_features.to(device=device, dtype=torch.float16)
        input_features = pad_trim_to_3000(input_features)

        with torch.no_grad():
            predicted_ids = gpu_models[gpu_id].generate(
                input_features,
                language="vi",
                task="transcribe"
            )

        texts = processor.batch_decode(predicted_ids, skip_special_tokens=True)
        texts = [t.strip() for t in texts]

        out = ["" for _ in wav_batch]
        for local_i, t in zip(valid_indices, texts):
            out[local_i] = t
        return out

    except RuntimeError as e:
        msg = str(e).lower()
        if ("out of memory" in msg or "cuda error" in msg) and len(wav_batch) > 1 and depth < 6:
            log(f"⚠️ OOM/CUDA on gpu:{gpu_id} batch={len(wav_batch)} → split & retry")
            try:
                torch.cuda.empty_cache()
            except Exception:
                pass
            mid = max(1, len(wav_batch)//2)
            left = transcribe_batch(gpu_id, wav_batch[:mid], depth=depth+1)
            right = transcribe_batch(gpu_id, wav_batch[mid:], depth=depth+1)
            return left + right
        raise
    except Exception as e:
        log(f"❌ transcribe_batch error (gpu:{gpu_id}): {type(e).__name__}: {e}")
        import traceback; traceback.print_exc()
        return ["" for _ in wav_batch]

def iter_L_folders(wav_root: str):
    Ls = [
        os.path.join(wav_root, d)
        for d in os.listdir(wav_root)
        if os.path.isdir(os.path.join(wav_root, d)) and d.startswith("L")
    ]
    return natsorted(Ls)

def process_L_folder(L_folder: str):
    folder_name = os.path.basename(L_folder)
    log("="*60)
    log(f"🎬 Processing {folder_name}")

    wav_files = natsorted([
        os.path.join(L_folder, f)
        for f in os.listdir(L_folder)
        if f.endswith(".wav")
    ])
    if not wav_files:
        log(f"⚠️ No WAV files in {folder_name}")
        return

    log(f"🎵 Found {len(wav_files)} WAV segments")

    results = {}
    batches = [wav_files[i:i+BATCH_SIZE] for i in range(0, len(wav_files), BATCH_SIZE)]
    log(f"📦 Total batches: {len(batches)} (BATCH_SIZE={BATCH_SIZE})")

    partial_path = os.path.join(OUTPUT_ROOT, f"_{folder_name}.partial.json")

    with ThreadPoolExecutor(max_workers=num_gpus) as ex:
        future_map = {}
        batch_info = {}
        for bi, batch in enumerate(batches):
            gpu_id = bi % num_gpus
            fut = ex.submit(transcribe_batch, gpu_id, batch)
            future_map[fut] = bi
            batch_info[bi] = batch

        pbar = tqdm(total=len(future_map), ncols=110, desc=f"🗣️ ASR {folder_name}")
        for fut in as_completed(future_map):
            bi = future_map[fut]
            batch = batch_info[bi]

            first_file = os.path.basename(batch[0])
            scene_range = f"{bi*BATCH_SIZE}-{bi*BATCH_SIZE + len(batch)-1}"
            pbar.set_postfix_str(f"Scenes {scene_range} | {first_file[:20]}")

            try:
                texts = fut.result()
                for i in range(len(batch)):
                    global_index = bi * BATCH_SIZE + i
                    results[str(global_index)] = texts[i] if i < len(texts) else ""
            except Exception as e:
                log(f"❌ batch {bi} error: {type(e).__name__}: {e}")
                import traceback; traceback.print_exc()
                for i in range(len(batch)):
                    global_index = bi * BATCH_SIZE + i
                    results[str(global_index)] = ""
            pbar.update(1)

            # checkpoint tạm mỗi 50 batches
            if (bi % 50) == 0:
                try:
                    with open(partial_path, "w", encoding="utf-8") as f:
                        json.dump(results, f, ensure_ascii=False)
                except Exception:
                    pass

        pbar.close()

    for idx in range(len(wav_files)):
        results.setdefault(str(idx), "")

    out_path = os.path.join(OUTPUT_ROOT, f"{folder_name}.json")
    log(f"💾 Saving to {out_path} ...")
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    try:
        if os.path.exists(partial_path):
            os.remove(partial_path)
    except Exception:
        pass

    log(f"✅ Saved: {out_path} | entries={len(results)} | non-empty={sum(1 for v in results.values() if v)}")


In [6]:
# =========================
# (5) Run
# =========================
if not os.path.exists(WAV_ROOT):
    raise RuntimeError(f"❌ WAV_ROOT not found: {WAV_ROOT}")

L_folders = iter_L_folders(WAV_ROOT)
if not L_folders:
    raise RuntimeError(f"❌ No L folders found in {WAV_ROOT}")

if RUN_FOLDER is not None:
    target = f"L{RUN_FOLDER:02d}"
    L_folders = [p for p in L_folders if os.path.basename(p) == target]
    if not L_folders:
        raise RuntimeError(f"❌ Folder {target} not found in {WAV_ROOT}")
    log(f"🎯 Running only folder: {target}")
else:
    log("📂 Running all L folders")

log(f"📂 Found folders: {[os.path.basename(x) for x in L_folders]}")
log(f"⚡ Using {num_gpus} GPU worker(s), BATCH_SIZE={BATCH_SIZE}")

for i, L_folder in enumerate(L_folders, 1):
    log(f"🎯 Progress: Folder {i}/{len(L_folders)}")
    process_L_folder(L_folder)

log("✅ All done.")


[19:58:51] 🎯 Running only folder: L24
[19:58:51] 📂 Found folders: ['L24']
[19:58:51] ⚡ Using 2 GPU worker(s), BATCH_SIZE=16
[19:58:51] 🎯 Progress: Folder 1/1
[19:58:51] ============================================================
[19:58:51] 🎬 Processing L24
[19:58:51] 🎵 Found 2347 WAV segments
[19:58:51] 📦 Total batches: 147 (BATCH_SIZE=16)


🗣️ ASR L24: 100%|██████████████████| 147/147 [10:57<00:00,  4.47s/it, Scenes 2240-2255 | L24_V043_scene005.wa]

[20:09:48] 💾 Saving to /kaggle/working/asr_json/L24.json ...
[20:09:48] ✅ Saved: /kaggle/working/asr_json/L24.json | entries=2347 | non-empty=2347
[20:09:48] ✅ All done.
